<p>What we want: P(Spam | message) = P(Spam | words)</p>
<p>Numerator: P(Spam) * P(words | Spam)</p>
<p>Denominator: P(Spam) * P(words | Spam) + P(not Spam) * P(words | not Spam)</p>
<p>my dictionary : call , free, now, get , bitcoin, send, new</p>

In [1]:
import pandas as pd 
import numpy as np

In [7]:
spamCollection =  pd.read_csv('Assets/SMSSpamCollection', sep = '\t' , header = None, names=['Label', 'SMS'])
spamCollectionRandom = spamCollection.sample(frac = 1, random_state= 1).reset_index(drop = True)

In [8]:
#splitting into training , validation and test sets 
training_test_index = round(len(spamCollectionRandom) * 0.8)
rest_index = len(spamCollectionRandom) - training_test_index

training_set = spamCollectionRandom[:training_test_index].reset_index(drop=True)
validation_set = spamCollectionRandom[training_test_index:training_test_index+ rest_index//2].reset_index(drop=True)
test_set = spamCollectionRandom[training_test_index + rest_index//2:].reset_index(drop=True)

In [9]:
training_set

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
4453,ham,"Sorry, I'll call later in meeting any thing re..."
4454,ham,Babe! I fucking love you too !! You know? Fuck...
4455,spam,U've been selected to stay in 1 of 250 top Bri...
4456,ham,Hello my boytoy ... Geeee I miss you already a...


In [15]:
training_set["Label"].value_counts()


ham     3858
spam     600
Name: Label, dtype: int64

In [17]:
numberOfSpam = 600 
numberOfNotSpam = 3858 
totalNumberOfMessages = len(training_set["Label"])

In [19]:
probabilityOfSpam = numberOfSpam/ totalNumberOfMessages
probabilityOfNotSpam = 1- probabilityOfSpam

<p>Here lets assume that the probability of every word appearing is independent in order for our model to run (Naive Bayes instead of Bayes)</p>
<p>Let's split our train dataset to calculate the probabilities</p>

In [30]:
spams = training_set[training_set["Label"] == "spam"].reset_index(drop= True)
notSpams = training_set[training_set["Label"] == "ham"].reset_index(drop= True)


In [31]:
spams

,Label,SMS
0,spam,FreeMsg Why haven't you replied to my text? I'...
1,spam,Congrats! 2 mobile 3G Videophones R yours. cal...
2,spam,FREE MESSAGE Activate your 500 FREE Text Messa...
3,spam,Call from 08702490080 - tells u 2 call 0906635...
4,spam,Someone has conacted our dating service and en...
...,...,...
595,spam,Congratulations YOU'VE Won. You're a Winner in...
596,spam,Win the newest “Harry Potter and the Order of ...
597,spam,Someone U know has asked our dating service 2 ...
598,spam,YOUR CHANCE TO BE ON A REALITY FANTASY SHOW ca...


In [32]:
notSpams

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
3853,ham,"How about clothes, jewelry, and trips?"
3854,ham,"Sorry, I'll call later in meeting any thing re..."
3855,ham,Babe! I fucking love you too !! You know? Fuck...
3856,ham,Hello my boytoy ... Geeee I miss you already a...


In [43]:
def remove_numeric(corpus):

    for index, sentence in enumerate(corpus):
        currentSentence = sentence 
        currentSentenceList = currentSentence.split()
        newSentence = filter(lambda x: x.isalpha(), currentSentenceList)
        joinedNewSentence = " ".join(newSentence)
        corpus[index] = joinedNewSentence

In [44]:
spamsList = list(spams["SMS"])
remove_numeric(spamsList)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

matrix = vectorizer.fit_transform(spamsList)
result = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
result


,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [46]:
vectorizer.get_feature_names()

['about',
 'abroad',
 'abta',
 'ac',
 'access',
 'accident',
 'accommodation',
 'accommodationvouchers',
 'account',
 'action',
 'activate',
 'active',
 'adam',
 'added',
 'address',
 'admirer',
 'ads',
 'adult',
 'advise',
 'advisors',
 'afraid',
 'after',
 'ag',
 'again',
 'age',
 'agent',
 'ages',
 'alerts',
 'alfie',
 'algarve',
 'all',
 'alone',
 'along',
 'already',
 'am',
 'amanda',
 'amazing',
 'among',
 'an',
 'and',
 'animation',
 'announcement',
 'anonymous',
 'another',
 'ans',
 'ansr',
 'answer',
 'any',
 'anyone',
 'anytime',
 'aom',
 'app',
 'apply',
 'approx',
 'arcade',
 'are',
 'area',
 'aries',
 'around',
 'arrange',
 'arrive',
 'arsenal',
 'as',
 'asian',
 'asked',
 'at',
 'attempt',
 'auction',
 'august',
 'availa',
 'available',
 'ave',
 'await',
 'awaiting',
 'award',
 'awarded',
 'away',
 'babe',
 'babes',
 'baby',
 'babygoodbye',
 'back',
 'backdoor',
 'bad',
 'badass',
 'bahamas',
 'bailiff',
 'balance',
 'band',
 'bangbabes',
 'banned',
 'barkleys',
 'bbdelux

In [49]:
#get index of all of our dictionary 


# dictionary = defaultdict(0)
dictionary = {"call": 0 , "free": 0, "now" : 0, "get": 0, "send": 0 , "new" : 0}
# since bitcoin is not in the dictionary, we apply laplace smoothing to make it a probability of 1/2 

for word in dictionary: 
    dictionary[word] = vectorizer.get_feature_names().index(word)

print(dictionary)


{'call': 146, 'free': 426, 'now': 785, 'get': 451, 'send': 1024, 'new': 765}


In [ ]:
# calculate the probabilty of word given that is a spam message
# P(spam) * P(every single dictionary | spam)

# P(every single dictionary | spam) = P(appearing) vs not appearing
# if a word didnt appear, give the default likelihood to be 1/2 (laplace smoothing)



